In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)

# Stellar mass distribution of satellites and isolated halos

Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (1,0)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

In [ ]:
z = data["LCDM"]["snapshot"].get_attribute("Redshift", "Header")

## Get data for plotting

Choose how to distinguish between satellites and isolated:

In [ ]:
distinction = "by_gn"

Read the datasets from each simulation, convert the units and split into satellites/isolated and luminous/dark:

In [ ]:
def arrange_sm_array(sm):
    sm = np.concatenate((np.sort(sm)[::-1], np.array([0.01]))) 
    return sm

def construct_count_array(size):
    return np.concatenate((np.arange(1, size+1), 
                           np.array([size])))

In [ ]:
for key, sim_data in data.items():    
    # Get data:
    snap = sim_data["snapshot"]
    sm = snap.get_subhalos("Stars/Mass") * units.g.to(units.Msun)
    
    # Add M31 and MW to the dictionary:
    idx_m31 = snap.index_of_halo(sim_data["M31_identifier"][0], sim_data["M31_identifier"][1])
    sim_data["M31"] = {"StellarMass": sm[idx_m31]}
    idx_mw = snap.index_of_halo(sim_data["MW_identifier"][0], sim_data["MW_identifier"][1])
    sim_data["MW"] = {"StellarMass": sm[idx_mw]}
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    if distinction == "by_r":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "by_gn":
        mask_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    
    mask_m31_lum = np.logical_and.reduce([mask_sat[0], mask_lum, mask_nonzero_vmax])
    mask_m31_dark = np.logical_and.reduce([mask_sat[0], mask_dark, mask_nonzero_vmax])
    mask_mw_lum = np.logical_and.reduce([mask_sat[1], mask_lum, mask_nonzero_vmax])
    mask_mw_dark = np.logical_and.reduce([mask_sat[1], mask_dark, mask_nonzero_vmax])
    mask_isol_lum = np.logical_and.reduce([mask_isol, mask_lum, mask_nonzero_vmax])
    mask_isol_dark = np.logical_and.reduce([mask_isol, mask_dark, mask_nonzero_vmax])
    
    # In case M31 and MW are identified as the same halo, remove intersection:
    mask_mw_lum = np.logical_and(mask_mw_lum, np.logical_not(mask_m31_lum))
    mask_mw_dark = np.logical_and(mask_mw_dark, np.logical_not(mask_m31_dark))
   
    # Add M31 satellites to the dictionary:
    sim_data["M31_satellites"] = \
    {"StellarMass": arrange_sm_array(sm[mask_m31_lum]),
     "SM_count": construct_count_array(np.sum(mask_m31_lum)),
     "N_dark": np.sum(mask_m31_dark)
    }
    
    # Add MW satellites to the dictionary:
    sim_data["MW_satellites"] = \
    {"StellarMass": arrange_sm_array(sm[mask_mw_lum]),
     "SM_count": construct_count_array(np.sum(mask_mw_lum)),
     "N_dark": np.sum(mask_mw_dark)
    }
        
    # Add all satellites combined to the dictionary:
    sim_data["all_satellites"] = \
    {"StellarMass": arrange_sm_array(
        np.concatenate([sm[mask_m31_lum], sm[mask_mw_lum]])),
     "SM_count": construct_count_array(
         np.sum(np.logical_or(mask_m31_lum, mask_mw_lum))),
     "N_dark": np.sum(np.logical_or(mask_m31_dark, mask_mw_dark))
    }
       
    # Add isolated galaxies to the dictionary:
    sim_data["isolated"] = \
    {"StellarMass": arrange_sm_array(sm[mask_isol_lum]),
     "SM_count": construct_count_array(np.sum(mask_isol_lum)),
     "N_dark": np.sum(mask_isol_dark)
    }

In [ ]:
# Set some parameters:
x_down = 10**5; x_up = 10**11
y_down = 1; y_up = 100

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = 'sat_and_isol_stellar_mass_dist_{}'.format(distinction)
for name in names:
    filename += "_{}".format(name)
filename += "_{}.png".format(snap_id)
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis:
for ax in axes:
    ax.set_xlim(x_down, x_up)
    ax.set_ylim(y_down, y_up)
    ax.set_xlabel('$M_*[M_\odot]$', fontsize=16)
    ax.set_ylabel('$N(>M_*)$', fontsize=16)
    ax.set_xscale('log')
    ax.set_yscale('log')

# Add redshift:
z_x = 10 ** (np.log10(x_up) - 9/10 * (np.log10(x_up)-np.log10(x_down)))
z_y = 10 ** (np.log10(y_up) - 1/10 * (np.log10(y_up)-np.log10(y_down)))
axes[0].text(z_x, z_y, "z = {:.2f}".format(z))
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['all_satellites']['StellarMass']
    y = entry['all_satellites']['SM_count']
    axes[0].plot(x, y, c=color[i], label=name)
    
    x = entry['isolated']['StellarMass']
    y = entry['isolated']['SM_count']
    axes[1].plot(x, y, c=color[i], label=name)
    
axes[0].legend(loc='upper right')
plt.tight_layout()

plt.savefig(filename, dpi=200)